# Customer segmentation

**RFM** approach to segment customers of an eshop into similar clusters using K-means, Agglomerative Clustering and Gaussian Mixture models.

**RFM** stands for:
- **R**ecency: Days since the customer made their last order.
- **F**requency: Total number of purchases made by the customer.
- **M**onetary: Sum of money the customer had spent in the eshop.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples
import seaborn as sns; sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
df = pd.read_csv("eshop.csv")
df.Date = pd.to_datetime(df.Date)

In [ ]:
df.head()

# RFM data frame

In [ ]:
today = df.Date.max()

rfm = df.groupby(['CustomerID']).agg({
        'Date': lambda x: (today - x.max()).days,
        'CustomerID': 'count', 
        'Subtotal': 'sum'})

rfm = rfm.rename({'Date': 'Recency', 'CustomerID': 'Frequency', 'Subtotal': 'Monetary'}, axis='columns')

In [ ]:
rfm.head()

In [ ]:
sns.pairplot(rfm)

### Note on scaling 
We need to scale the dataset in order to reduce great differences between variables (eg frequency will almost always be many times smaller than recency or monetary). I decided to go with the standard scaler, as it produced best results.

In [ ]:
distortions = []
K = range(1,20)
for k in K:
    pipeline = Pipeline([('scaler', StandardScaler()), ('clusterer', KMeans(n_clusters=k))])
    pipeline.fit(rfm.values)
    distortions.append(pipeline[-1].inertia_)

chart = sns.lineplot(x=K, y=distortions)
chart.set_title('Elbow method for getting K')
chart.set_xlabel('Number of clusters - K')
chart.set_ylabel('Score')
plt.show()

Number of clusters can be decided using many methods, such as the elbow method. In our case it seems the "elbow" is with `K = 3`, so I'll go with it and see.

In [ ]:
K = 3

### KMeans

In [ ]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clusterer', KMeans(n_clusters=K))])
pipeline.fit(rfm.values)

In [ ]:
kmeans = pipeline[-1]

In [ ]:
clustered_rfm = rfm.copy(deep=True)
clustered_rfm['CustomerCategory'] = pipeline[-1].labels_

In [ ]:
clustered_rfm.head()

In [ ]:
clusters = []

for i in range(pipeline[-1].n_clusters):
    description = clustered_rfm[clustered_rfm.CustomerCategory == i].describe()
    clusters.append(description)
    display(description)

In [ ]:
superstars_id = max(zip(map(lambda cluster: cluster.loc['mean', 'Monetary'], clusters), range(len(clusters))))[1]
losers_id = max(zip(map(lambda cluster: cluster.loc['mean', 'Recency'], clusters), range(len(clusters))))[1]
normies_id = list(set([0, 1, 2]) - set([superstars_id, losers_id]))[0]

print(f'Superstar customers cluster: {superstars_id}.')
print(f'Normal customers cluster: {normies_id}.')
print(f'Uninteresting customers cluster: {losers_id}.')

## Agglomerative clustering

In [ ]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clusterer', AgglomerativeClustering(n_clusters=K))])
pipeline.fit(rfm.values)

In [ ]:
clustered_rfm = rfm.copy(deep=True)
clustered_rfm['CustomerCategory'] = pipeline[-1].labels_

In [ ]:
clustered_rfm.head()

In [ ]:
clusters = []

for i in range(pipeline[-1].n_clusters):
    description = clustered_rfm[clustered_rfm.CustomerCategory == i].describe()
    clusters.append(description)
    display(description)

In [ ]:
superstars_id = max(zip(map(lambda cluster: cluster.loc['mean', 'Monetary'], clusters), range(len(clusters))))[1]
losers_id = max(zip(map(lambda cluster: cluster.loc['mean', 'Recency'], clusters), range(len(clusters))))[1]
normies_id = list(set([0, 1, 2]) - set([superstars_id, losers_id]))[0]

print(f'Superstar customers cluster: {superstars_id}.')
print(f'Normal customers cluster: {normies_id}.')
print(f'Uninteresting customers cluster: {losers_id}.')

## Gaussian mixture
I'm using this algorithm just out of curiosity to see if the customers are "generated" with respect to Gaussian distribution.

In [ ]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clusterer', GaussianMixture(n_components=K))])
pipeline.fit(rfm.values)

In [ ]:
clustered_rfm = rfm.copy(deep=True)
clustered_rfm['CustomerCategory'] = pipeline[-1].predict(rfm.values)

In [ ]:
clustered_rfm.head()

In [ ]:
clusters = []

for i in range(pipeline[-1].n_components):
    description = clustered_rfm[clustered_rfm.CustomerCategory == i].describe()
    clusters.append(description)
    display(description)

We can clearly see that the `GaussianMixture` doesn't work in this case (most likely because customers are not generated from Gaussian distribution). I was just curious to see.

## Summary
Turns out that both `KMeans` and `AgglomerativeClustering` work well in this case and are able to correctly identify all `3` clusters correctly (more or less), while `GaussianMixture` only identified `2` clusters (still correctly, but `2` none the less...)

# Silhouette analysis

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# borrowed from https://gist.github.com/clintval/e9afc246e77f6488cda79f86e4d37148
def silhouette_plot(X, y, n_clusters, ax=None):
    from sklearn.metrics import silhouette_samples, silhouette_score

    if ax is None:
        ax = plt.gca()

    # Compute the silhouette scores for each sample
    silhouette_avg = silhouette_score(X, y)
    sample_silhouette_values = silhouette_samples(X, y)

    y_lower = padding = 2
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        ith_cluster_silhouette_values = sample_silhouette_values[y == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        cmap = cm.get_cmap("tab10")
        color = cmap(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                         0,
                         ith_cluster_silhouette_values,
                         facecolor=color,
                         edgecolor=color,
                         alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + padding

    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhoutte score of all the values
    ax.axvline(x=silhouette_avg, c='r', alpha=0.8, lw=0.8, ls='-')
    ax.annotate('Average',
                xytext=(silhouette_avg, y_lower * 1.025),
                xy=(0, 0),
                ha='center',
                alpha=0.8,
                c='r')

    ax.set_yticks([])  # Clear the yaxis labels / ticks
    ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_ylim(0, y_upper + 1)
    ax.set_xlim(-0.075, 1.0)
    ax.figure.set_size_inches(10, 15)
    return ax

In [ ]:
silhouette_plot(rfm.values, kmeans.labels_, n_clusters=3)

## Summary
From the silhouette chart it can be seen that as predicted the cluster of "superstars" is smallest one, normal customers cluster is the greatest and uninteresting are somewhere in between.

Given the fact how many elements in each cluster have silhouette score smaller than 0, we can suppose that the number of clusters was chosen correctly. Only the cluster of "superstars" has more elements with silhouette score smaller than zero, than the others.